In [1]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import uuid
import os
import urllib


class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, Postcode = "BT4"):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(Postcode).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

##
# NAVIGATE THE WEBSITE
##

if __name__ == "__main__":
    bot = Scraper()
    bot.ivan_accept_cookies()
    time.sleep(2)
    bot.button_click('//a[@href="/login"]')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Password"]','sopranotony321')
    time.sleep(2)
    bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
    time.sleep(2)
    bot.button_click('//a[@class="mainnav-logo"]')  
    time.sleep(2)
    bot.search_word('//*[@id="searchForm"]/div/div[1]')
    time.sleep(2)
    bot.search_rent()
    time.sleep(2)
    

    #CREATE THE LIST OF LINKS
    list_links = []
    img_links = []
    print("Finding elements...")
    while True:
        container = bot.find_container()
        items = container.find_elements(By.XPATH, './li')
        for i in items:
            try:
                house = i.find_element(By.XPATH, './/a[2]')
                link = house.get_attribute('href')
                list_links.append(link)
                time.sleep(1)
                print('looking for container')
                container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
                print('found container')
                items_2 = container_2.find_elements(By.XPATH, './/img')
                print('found items')
                for i in items_2:
                    try:
                        link = i.get_attribute('src')
                        img_links.append(link)
                    except:
                        print('No src found')
            except NoSuchElementException:
                print('No element found')
            except Exception as e:
                print(e)
        
        try:
            bot.button_click('//a[@class="btn paging-next"]')
        except NoSuchElementException:
            print("No more pages")
            break

    #GRAB INFO FROM EACH LINK AND STORE
    print("gathering info...")
    prop_dict = {"fr-id": [],
            "id": [],
            "Link": [],
             "Summary": [],
            "Address": [],
            "Price": [],
            "Image links": []
            }
    for link in list_links:
        im = link[-6:]
        im2 = link[28:35]
        prop_dict["fr-id"].append(im2+im)
        id = uuid.uuid4()
        prop_dict["id"].append(id)
        bot.driver.get(link)
        prop_dict["Link"].append(link)
        time.sleep(1)
        summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
        prop_dict["Summary"].append(summary.text)
        time.sleep(1)
        info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
        address = info.find_element(By.XPATH, './/h1')
        prop_dict["Address"].append(address.text)
        time.sleep(1)
        price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
        prop_dict["Price"].append(price.text)
        prop_dict["Image links"].append(img_links)


    Postcode = "BT4"
    os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}")
    image_directory = os.path.dirname(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}/")
    img_link_ct=0
    for i in prop_dict["Image links"]:
        img_link_ct += 1
        img_ct = 0
        for url in i:
            try:
                img_ct += 1
                req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                with open(f"{image_directory}/{prop_dict['fr-id'][img_link_ct]}_{str(img_ct)}.jpg", "wb") as f:
                    with urllib.request.urlopen(req) as r:
                        f.write(r.read())
            except IndexError:
                pass

df = pd.DataFrame(prop_dict)
df

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_76763/3368490567.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_76763/3368490567.py:59: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_76763/3368490567.py:51: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="s

Finding elements...
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
looking for container
No element found
No element found
No more pages
gathering info...


,fr-id,id,Link,Summary,Address,Price,Image links
0,56-park766769,01ef3a17-4771-4ec9-a148-0bbd60cb6716,https://www.propertypal.com/56-park-avenue-bel...,3 Bed Terrace House To Rent,"56 PARK AVENUE,\nBELFAST BT4 1JJ",£795\n/ month + fees,[]
1,apt-1-2548636,c58afd82-05ee-4ea1-a8f7-79bf69ffcc86,https://www.propertypal.com/apt-1-223d-belmont...,1 Bed Apartment To Rent,"APT 1, 223D BELMONT ROAD,\nBELFAST BT4 2AH",£725\n/ month + fees,[]
2,1a-care765807,9cc989cc-5893-47ec-bdf9-ad4224e1e598,https://www.propertypal.com/1a-carew-street-be...,2 Bed End-terrace House To Rent,"1A CAREW STREET,\nBelfast BT4 1HQ",£600\n/ month,[]
3,13-bath765788,189cf83d-2e67-4510-99f1-0eb78d593e00,https://www.propertypal.com/13-bathgate-drive-...,4 Bed Terrace House To Rent,"13 BATHGATE DRIVE,\nBelfast BT4 2BA",£895\n/ month + fees,[]
4,flat-3-532728,38b6501b-b3d3-4ebb-b087-e55a0a0f79cc,https://www.propertypal.com/flat-3-152-upper-n...,1 Bed Apartment To Rent,"FLAT 3, 152 UPPER NEWTOWNARDS ROAD,\n152 Upper...",£550\n/ month + fees,[]
5,229-par765499,b63b85c8-5e25-470d-be73-81b01243c28b,https://www.propertypal.com/229-park-avenue-be...,4 Bed Mid-terrace House To Rent,"229 PARK AVENUE,\nBelfast BT4 1LN","£1,100\n/ month + fees",[]
6,9-pims-765427,fd7e2892-4fd6-451c-aafe-ca57dcd85050,https://www.propertypal.com/9-pims-avenue-belf...,3 Bed Terrace House To Rent,"9 PIMS AVENUE,\nBELFAST BT4 1PJ",£750\n/ month + fees,[]
7,apt-3-g765358,a0fd694a-1f65-4618-a95c-ee3b88281dcb,https://www.propertypal.com/apt-3-goldstone-ha...,2 Bed 1st Floor Apartment To Rent,"APT 3, GOLDSTONE HALL,\nBelfast BT4 1JE",£775\n/ month + fees,[]
8,53-uppe765329,e6fa9a92-cd3c-47c2-babf-af66fd39e97b,https://www.propertypal.com/53-upper-newtownar...,3 Bed Terrace House To Rent,"53 UPPER NEWTOWNARDS ROAD,\nBELFAST BT4 3HT",£850\n/ month + fees,[]
9,4-devon765273,b4615223-ff40-4e9c-93fb-98cea4914d92,https://www.propertypal.com/4-devon-drive-syde...,2 Bed Terrace House To Rent,"4 DEVON DRIVE,\nSydenham, Belfast BT4 1LF",£575\n/ month,[]


In [3]:
bot = Scraper()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_76763/3368490567.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [2]:
container = bot.find_container()
items = container.find_elements(By.XPATH, './li')
for i in items:
    try:
        house = i.find_element(By.XPATH, './/a[2]')
        link = house.get_attribute('href')
        list_links.append(link)
        time.sleep(1)
        print('looking for container')
        container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
        print('found container')
        items_2 = container_2.find_elements(By.XPATH, './/img')
        print('found items')
        for i in items_2:
            try:
                link = i.get_attribute('src')
                img_links.append(link)
            except:
                print('No src found')
    except NoSuchElementException:
        print('No element found')
    except Exception as e:
        print(e)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_76763/3368490567.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="body"]/div[3]/div/div[1]/div/ul"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x000000010397c079 chromedriver + 4444281
1   chromedriver                        0x0000000103908403 chromedriver + 3970051
2   chromedriver                        0x00000001035a3038 chromedriver + 409656
3   chromedriver                        0x00000001035d92fd chromedriver + 631549
4   chromedriver                        0x00000001035d9581 chromedriver + 632193
5   chromedriver                        0x000000010360b4f4 chromedriver + 836852
6   chromedriver                        0x00000001035f67bd chromedriver + 751549
7   chromedriver                        0x0000000103609221 chromedriver + 827937
8   chromedriver                        0x00000001035f6683 chromedriver + 751235
9   chromedriver                        0x00000001035cca45 chromedri

In [17]:
class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, text):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(text).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()
    
    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

In [19]:
bot = Scraper()
bot.ivan_accept_cookies()
time.sleep(2)
bot.search_word('//*[@id="searchForm"]/div/div[1]', 'bt4')
time.sleep(2)
bot.search_rent()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')


In [59]:
bot = Scraper()
bot.ivan_accept_cookies()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [53]:
bot = Scraper()
bot.ivan_accept_cookies()
bot.button_click('//a[@href="/login"]')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


In [54]:
bot = Scraper()
bot.ivan_accept_cookies()
bot.button_click('//a[@href="/login"]')
bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')

In [113]:
bot = Scraper()
bot.ivan_accept_cookies()
time.sleep(2)
bot.button_click('//a[@href="/login"]')
time.sleep(2)
bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
time.sleep(2)
bot.search_word('//input[@placeholder="Password"]','sopranotony321')
time.sleep(2)
bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
time.sleep(2)
bot.button_click('//a[@class="mainnav-logo"]')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


In [56]:
bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


In [57]:
bot.button_click('//a[@class="mainnav-logo"]')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


In [114]:
container = bot.find_container()

In [115]:
items = container.find_elements(By.XPATH, './li')
print(items)
list_links = []

[<selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="79985579-5e29-499b-81dd-90b7bf676e16")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="aff8746d-55b3-4408-81ae-a4a4ce27a24c")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="e87a33e9-4504-4eae-a587-5bd519750a56")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="36e06693-2224-4d0c-9163-4912a28d9000")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="dd746f6a-6989-4e5a-99d0-36f6ef43f081")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="f4ab1e6d-a5e8-4f1e-9d22-02c741466b7e")>, <selenium.webdriver.remote.webelement.WebElement (session="b6a148851ce2e9ac7589947b2f3fb558", element="69fa86c3-b21c-49a2-a878-c3559d00

In [116]:

house = items.find_element(By.XPATH, './/a[2]')
link = house.get_attribute('href')
print(link)

AttributeError: 'list' object has no attribute 'find_element'

In [133]:
 for i in items:
    try:
        house = i.find_element(By.XPATH, './/a[2]')
        link = house.get_attribute('href')
        print(link)
        list_links.append(link)
    except:
        print('No href found, skipping this property')
 print(list_links)

No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
['https://www.propertypal.com/apt-14-39-dundela-crescent-belfast/759335', 'https://www.propertypal.com/1a-carew-street-belfast/765807', 'https://www.propertypal.com/13-bathgate-drive-belfast/765788', 'https://www.propertypal.com/flat-3-152-upper-newtownards-road-152-upper-newtownards-road-belfast/532728', 'https://www.propertypal.com/229-park-avenue-belfast/765499', 'https://www.propertypal.com/9-pims-avenue-belfast/765427', 'https://www.propertypal.com/apt-3-goldstone-hall-belfast/765358', 'https://www.propertypal.com/53-upper-newtownards-road-belfast/765329', 'https://www.p

In [118]:
prop_dict = {"Link": [],
             "Summary": [],
            "Address": []
            "Price": []
            }
for link in list_links:
    bot.driver.get(link)
    prop_dict["Link"].append(link)
    time.sleep(1)
    summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
    prop_dict["Summary"].append(summary.text)
    time.sleep(1)
    info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
    address = info.find_element(By.XPATH, './/h1')
    prop_dict["Address"].append(address.text)
    time.sleep(1)
    price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
    prop_dict["Price"].append(price)
    
print(prop_dict)

{'Link': ['https://www.propertypal.com/apt-14-39-dundela-crescent-belfast/759335', 'https://www.propertypal.com/1a-carew-street-belfast/765807', 'https://www.propertypal.com/13-bathgate-drive-belfast/765788', 'https://www.propertypal.com/flat-3-152-upper-newtownards-road-152-upper-newtownards-road-belfast/532728', 'https://www.propertypal.com/229-park-avenue-belfast/765499', 'https://www.propertypal.com/9-pims-avenue-belfast/765427', 'https://www.propertypal.com/apt-3-goldstone-hall-belfast/765358', 'https://www.propertypal.com/53-upper-newtownards-road-belfast/765329', 'https://www.propertypal.com/4-devon-drive-sydenham-belfast/765273', 'https://www.propertypal.com/40-norwood-avenue-belfast/765201'], 'Summary': ['2 Bed Apartment To Rent', '2 Bed End-terrace House To Rent', '4 Bed Terrace House To Rent', '1 Bed Apartment To Rent', '4 Bed Mid-terrace House To Rent', '3 Bed Terrace House To Rent', '2 Bed 1st Floor Apartment To Rent', '3 Bed Terrace House To Rent', '2 Bed Terrace House To

In [120]:
import pandas as pd

pd.DataFrame(prop_dict)

,Link,Summary,Address
0,https://www.propertypal.com/apt-14-39-dundela-...,2 Bed Apartment To Rent,"APT 14, 39 DUNDELA CRESCENT,\nBelfast BT4 3BN"
1,https://www.propertypal.com/1a-carew-street-be...,2 Bed End-terrace House To Rent,"1A CAREW STREET,\nBelfast BT4 1HQ"
2,https://www.propertypal.com/13-bathgate-drive-...,4 Bed Terrace House To Rent,"13 BATHGATE DRIVE,\nBelfast BT4 2BA"
3,https://www.propertypal.com/flat-3-152-upper-n...,1 Bed Apartment To Rent,"FLAT 3, 152 UPPER NEWTOWNARDS ROAD,\n152 Upper..."
4,https://www.propertypal.com/229-park-avenue-be...,4 Bed Mid-terrace House To Rent,"229 PARK AVENUE,\nBelfast BT4 1LN"
5,https://www.propertypal.com/9-pims-avenue-belf...,3 Bed Terrace House To Rent,"9 PIMS AVENUE,\nBELFAST BT4 1PJ"
6,https://www.propertypal.com/apt-3-goldstone-ha...,2 Bed 1st Floor Apartment To Rent,"APT 3, GOLDSTONE HALL,\nBelfast BT4 1JE"
7,https://www.propertypal.com/53-upper-newtownar...,3 Bed Terrace House To Rent,"53 UPPER NEWTOWNARDS ROAD,\nBELFAST BT4 3HT"
8,https://www.propertypal.com/4-devon-drive-syde...,2 Bed Terrace House To Rent,"4 DEVON DRIVE,\nSydenham, Belfast BT4 1LF"
9,https://www.propertypal.com/40-norwood-avenue-...,3 Bed Semi-detached House To Rent,"40 NORWOOD AVENUE,\nBelfast BT4 2EF"


In [137]:
list_links = []
while True:
    container = bot.find_container()
    items = container.find_elements(By.XPATH, './li')
    for i in items:
        try:
            house = i.find_element(By.XPATH, './/a[2]')
            link = house.get_attribute('href')
            list_links.append(link)
        except:
            print('No href found, skipping this property')
    
    try:
        bot.button_click('//a[@class="btn paging-next"]')
    except NoSuchElementException:
        print("end of list")
        break


No href found, skipping this property


/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
end of list


In [138]:
print(list_links)
len(list_links)

['https://www.propertypal.com/apt-14-39-dundela-crescent-belfast/759335', 'https://www.propertypal.com/1a-carew-street-belfast/765807', 'https://www.propertypal.com/13-bathgate-drive-belfast/765788', 'https://www.propertypal.com/flat-3-152-upper-newtownards-road-152-upper-newtownards-road-belfast/532728', 'https://www.propertypal.com/229-park-avenue-belfast/765499', 'https://www.propertypal.com/9-pims-avenue-belfast/765427', 'https://www.propertypal.com/apt-3-goldstone-hall-belfast/765358', 'https://www.propertypal.com/53-upper-newtownards-road-belfast/765329', 'https://www.propertypal.com/4-devon-drive-sydenham-belfast/765273', 'https://www.propertypal.com/40-norwood-avenue-belfast/765201', 'https://www.propertypal.com/the-limes-216-belmont-road-belfast/105704', 'https://www.propertypal.com/apt-14-39-dundela-crescent-belfast/759335', 'https://www.propertypal.com/35-sydenham-avenue-belmont-belfast/764757', 'https://www.propertypal.com/79-upper-newtownards-road-belfast/730603', 'https:/

23

In [123]:
bot = Scraper()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [126]:
bot.button_click('//a[@class="btn paging-next"]')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_6413/2763093461.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class="btn paging-next"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x00000001083c5079 chromedriver + 4444281
1   chromedriver                        0x0000000108351403 chromedriver + 3970051
2   chromedriver                        0x0000000107fec038 chromedriver + 409656
3   chromedriver                        0x00000001080222fd chromedriver + 631549
4   chromedriver                        0x0000000108022581 chromedriver + 632193
5   chromedriver                        0x00000001080544f4 chromedriver + 836852
6   chromedriver                        0x000000010803f7bd chromedriver + 751549
7   chromedriver                        0x0000000108052221 chromedriver + 827937
8   chromedriver                        0x000000010803f683 chromedriver + 751235
9   chromedriver                        0x0000000108015a45 chromedriver + 58016

In [109]:
summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
print(summary.text)

2 Bed Apartment To Rent


In [99]:
info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')

address = info.find_element(By.XPATH, './/h1')
print(address.text)
price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
print(price.text)
summary = bot.driver.find_element(By.XPATH, '//p[@class="prop-strapline"]')
print(summary.text)

1A CAREW STREET,
Belfast BT4 1HQ
£600
/ month



In [102]:
summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
print(summary.text)

2 Bed End-terrace House To Rent


In [97]:
info_3 = bot.driver.find_element(By.NAME, 'Bedrooms')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="Bedrooms"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x000000010d7f8079 chromedriver + 4444281
1   chromedriver                        0x000000010d784403 chromedriver + 3970051
2   chromedriver                        0x000000010d41f038 chromedriver + 409656
3   chromedriver                        0x000000010d4552fd chromedriver + 631549
4   chromedriver                        0x000000010d455581 chromedriver + 632193
5   chromedriver                        0x000000010d4874f4 chromedriver + 836852
6   chromedriver                        0x000000010d4727bd chromedriver + 751549
7   chromedriver                        0x000000010d485221 chromedriver + 827937
8   chromedriver                        0x000000010d472683 chromedriver + 751235
9   chromedriver                        0x000000010d448a45 chromedriver + 580165
10 

for link in list_links:
    time.sleep(3)
    bot.driver.get(link)

In [25]:
bot = Scraper()
time.sleep(3)
bot.accept_cookies('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
bot.search_word('//*[@id="searchForm"]/div/div[1]', 'BT4')
bot.search_rent()
time.sleep(2)
bot.scroll_down()
bot.scroll_down()
bot.button_click('//*[@id="body"]/div[3]/div/div[1]/div/ul/li[1]/div/a[2]')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_24373/3071741555.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_24373/3071741555.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_24373/3071741555.py:41: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.dr

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.propertypal.com")
time.sleep(2)
accept_cookies = driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
accept_cookies.click()
time.sleep(2)
search = driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/div[1]')
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(search).click(search).perform()
ActionChains(driver).send_keys("BT4").perform()

In [77]:
propertypal = Scraper("https://www.propertypal.com")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_82170/937025597.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [78]:
propertypal.accept_cookies('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_82170/937025597.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)


In [36]:
propertypal.search_word('//*[@id="searchForm"]/div/div[1]','BT4 BT5')

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_82170/2345440001.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_bar = self.driver.find_element_by_xpath(xpath)


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=103.0.5060.53)
Stacktrace:
0   chromedriver                        0x0000000100d45079 chromedriver + 4444281
1   chromedriver                        0x0000000100cd1403 chromedriver + 3970051
2   chromedriver                        0x000000010096beef chromedriver + 409327
3   chromedriver                        0x000000010099c60f chromedriver + 607759
4   chromedriver                        0x000000010099bc0d chromedriver + 605197
5   chromedriver                        0x00000001009bf792 chromedriver + 751506
6   chromedriver                        0x0000000100996f65 chromedriver + 585573
7   chromedriver                        0x00000001009bf89e chromedriver + 751774
8   chromedriver                        0x00000001009d2221 chromedriver + 827937
9   chromedriver                        0x00000001009bf683 chromedriver + 751235
10  chromedriver                        0x0000000100995a45 chromedrive

In [81]:
search = propertypal.driver.find_element(By.XPATH, '//*[@id="query"]')
search.click()
search.send_keys("BT4")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=103.0.5060.53)
Stacktrace:
0   chromedriver                        0x000000010c4c8079 chromedriver + 4444281
1   chromedriver                        0x000000010c454403 chromedriver + 3970051
2   chromedriver                        0x000000010c0eeeef chromedriver + 409327
3   chromedriver                        0x000000010c126477 chromedriver + 636023
4   chromedriver                        0x000000010c11a6a9 chromedriver + 587433
5   chromedriver                        0x000000010c142792 chromedriver + 751506
6   chromedriver                        0x000000010c119f65 chromedriver + 585573
7   chromedriver                        0x000000010c14289e chromedriver + 751774
8   chromedriver                        0x000000010c155221 chromedriver + 827937
9   chromedriver                        0x000000010c142683 chromedriver + 751235
10  chromedriver                        0x000000010c118a45 chromedrive

In [72]:
search_bar = propertypal.driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/div[1]/section')
search_bar.click
search_bar.send_keys("BT4")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=103.0.5060.53)
Stacktrace:
0   chromedriver                        0x0000000101cd0079 chromedriver + 4444281
1   chromedriver                        0x0000000101c5c403 chromedriver + 3970051
2   chromedriver                        0x00000001018f6eef chromedriver + 409327
3   chromedriver                        0x000000010192760f chromedriver + 607759
4   chromedriver                        0x0000000101926c0d chromedriver + 605197
5   chromedriver                        0x000000010194a792 chromedriver + 751506
6   chromedriver                        0x0000000101921f65 chromedriver + 585573
7   chromedriver                        0x000000010194a89e chromedriver + 751774
8   chromedriver                        0x000000010195d221 chromedriver + 827937
9   chromedriver                        0x000000010194a683 chromedriver + 751235
10  chromedriver                        0x0000000101920a45 chromedrive

In [65]:
propertypal.scroll_down()

In [82]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time 

In [83]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.propertypal.com")
time.sleep(2)
accept_cookies = driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
accept_cookies.click()
time.sleep(2)
search = driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/div[1]')
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(search).click(search).perform()
ActionChains(driver).send_keys("BT4").perform()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_82170/199523797.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
